[Rendered Version of this notebook](http://nbviewer.jupyter.org/github/andersonfrailey/Notebook-Uploads/blob/master/CPS%20Comparisons.ipynb)

In [89]:
from taxcalc import *
from taxcalc.utils import *
from bokeh.io import show, output_notebook
from bokeh.charts import Bar, Scatter
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.models import HoverTool
from collections import OrderedDict
import copy
from notebookfunctions import distribution, index_list, percentile
output_notebook()
import json

Loading BokehJS ...

### Missing Variables

In [90]:
cps = pd.read_csv('../taxdata/cps_data/cps_ben_full.csv')
usable = open('taxcalc/records_variables.json')
usable_vars = json.load(usable)
usable.close()
missing = 0
for item in usable_vars['read'].keys():
    if item not in cps.columns:
        print '{}: {}'.format(item, usable_vars['read'][item]['desc'])
        missing += 1
print '\nTotal Variables Missing: {}'.format(missing)

p23250: Sch D: Net long-term capital gains/losses
p25470: Sch E: Royalty depletion and/or rental depreciation
e09800: Unreported payroll taxes from Form 4137 or 8919
e02000: Sch E rental, royalty, S-corp, etc, income/loss
e62900: Alternative Minimum Tax foreign tax credit from Form 6251
p08000: Other tax credits (but not including Sch R credit)
e58990: Investment income elected amount from Form 4952
e00700: Taxable refunds of state and local income taxes
e03290: Health savings account deduction from Form 8889
e07240: Retirement savings contributions credit from Form 8880
e27200: Sch E: Farm rent net income or loss
e01200: Other net gain/loss from Form 4797
e03500: Alimony paid
e07260: Residential energy credit from Form 5695
p22250: Sch D: Net short-term capital gains/losses
e03220: Educator expenses
e07400: General business credit from Form 3800
f6251: 1 if Form 6251 (AMT) attached to return; otherwise 0
e03230: Tuition and fees from Form 8917
e03400: Penalty on early withdrawal of sa

In [91]:
# Data from IRS-SOI Tax Stats. Used for comparison
soi_stats = pd.read_csv('soi_stats.csv', index_col=0)  # Equivalent to tax-calc diagnostic table
soi_income = pd.read_csv('soi_income_stats.csv')  # Distribution of income items
soi_deductions = pd.read_csv('soi_deductions.csv', index_col=0)  # Itemized deductions
soi_deductions['index'] = soi_deductions.index

In [92]:
# Create calculator from the PUF
calc_puf = Calculator(records=Records(), policy=Policy())
calc_puf.advance_to_year(2014)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [93]:
# Calculator the CPS
wt = pd.read_csv('../Dropbox/cps_weights.csv')
# wt.drop('SEQUENCE', inplace=True, axis=1)
# wt = wt.loc[:,:'WT2026']
wt *= 100
recs_cps = Records(data=cps,
                   weights=wt,
                   adjust_ratios=None,
                   start_year=2014)
pol_cps = Policy(start_year=2014, num_years=11)
calc_cps = Calculator(records=recs_cps, policy=pol_cps)
# calc_cps.advance_to_year(2014)

In [94]:
calc_puf.calc_all()
calc_cps.calc_all()

### CPS Distribution Table

In [95]:
create_distribution_table(calc_cps.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,-922,"16,318,932","-1,104","7,845,226","6,691","6,115",3,"5,127",7,0,"-1,107",0,0,0,0,0,20,-20,78,58
1,"7,071","16,320,046","4,685","13,182,226","7,270","76,949",46,"5,355",83,4,"4,662","11,053",0,4,1,0,620,-617,562,-55
2,"14,371","16,319,101","8,620","11,922,137","7,440","1,065,169",788,"6,028","1,143",107,"8,209","34,152",1,108,20,0,"1,046",-958,"1,119",161
3,"21,307","16,319,736","14,445","12,209,370","7,241","2,536,210","1,673","6,628","3,816",383,"13,474","30,776",2,385,65,0,"1,197",-878,"1,953","1,075"
4,"28,971","16,319,946","22,057","11,162,595","6,777","4,552,338","3,122","7,176","8,277",936,"20,130","34,817",3,939,133,0,"1,059",-254,"3,035","2,782"
5,"38,690","16,319,325","31,119","9,994,923","6,310","6,082,204","4,778","7,651","14,568","1,724","28,189","24,142",4,"1,728",224,0,630,875,"4,311","5,186"
6,"51,890","16,319,456","44,334","8,136,960","5,246","8,095,572","7,542","8,048","24,485","3,054","39,690","28,913",3,"3,057",342,0,185,"2,530","6,083","8,613"
7,"71,513","16,319,345","64,777","5,772,933","3,736","10,539,232","11,587","8,811","40,967","5,714","57,781","24,864",4,"5,717",495,0,30,"5,192","8,857","14,049"
8,"103,373","16,320,043","95,233","3,544,070","2,385","12,773,616","16,539","9,920","66,512","9,926","85,644","37,716",5,"9,932",596,0,5,"9,331","13,113","22,444"
9,"245,849","16,319,840","232,288","1,455,319","1,044","14,863,980","27,026","9,761","194,523","44,247","218,286","2,571,788",601,"44,848",119,375,0,"45,103","21,899","67,002"


### PUF Distribution Table

In [96]:
create_distribution_table(calc_puf.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"-6,904","16,222,114","-7,252","11,094,216","5,964","6,150",18,"4,135",44,4,"-7,601",0,0,4,0,6,141,-131,353,222
1,"8,443","16,220,773","6,016","13,627,903","7,057","48,948",29,"4,913",262,23,"5,990","20,950",0,24,0,4,774,-747,821,74
2,"15,025","16,223,499","10,098","13,243,287","7,608","279,753",180,"6,493","1,482",145,"9,974","27,189",1,146,19,6,"1,650","-1,517","1,428",-89
3,"21,963","16,222,202","15,120","14,169,419","7,787","683,754",548,"6,822","3,991",420,"14,767","31,485",2,422,82,13,"1,460","-1,107","2,039",932
4,"30,413","16,223,284","22,440","14,142,144","7,921","1,516,658","1,287","7,187","8,668",999,"21,647","18,616",2,"1,002",217,17,"1,067",-265,"2,881","2,615"
5,"40,936","16,222,555","32,564","13,206,631","7,687","2,863,096","2,661","7,339","16,532","2,017","30,986","15,840",2,"2,020",362,26,548,"1,136","3,965","5,101"
6,"54,706","16,222,223","46,478","11,339,850","7,067","4,849,894","5,036","7,734","27,243","3,573","43,467","10,617",2,"3,575",506,42,164,"2,947","5,336","8,283"
7,"74,668","16,222,702","67,782","9,028,824","6,082","7,185,021","8,156","8,574","45,098","6,519","63,224","46,267",5,"6,524",681,62,80,"5,825","7,708","13,533"
8,"110,558","16,222,104","102,839","5,871,767","4,308","10,349,258","13,730","10,142","74,728","11,702","95,552","184,541",18,"11,720",889,100,95,"10,835","12,479","23,314"
9,"338,763","16,223,164","321,658","2,019,136","1,497","14,190,613","36,479","9,580","274,512","69,165","306,721","4,448,053","2,351","71,516","1,789","2,138",55,"71,811","21,957","93,768"


### Diagnostic Table Comparison

In [97]:
cps_diag = create_diagnostic_table(calc_cps)

In [98]:
puf_diag = create_diagnostic_table(calc_puf)

In [99]:
diag_data = pd.DataFrame()
diag_data['SOI'] = soi_stats['Value']
diag_data['CPS'] = cps_diag[2014]
diag_data['PUF'] = puf_diag[2014]
diag_data['% Change'] = ((cps_diag[2014] / puf_diag[2014]) - 1) * 100

In [100]:
diag_data

,SOI,CPS,PUF,% Change
Returns (#m),148.6,163.2,162.2,0.6
AGI ($b),"9,771.00","8,428.4","10,021.5",-15.9
Itemizers (#m),44,60.6,42.0,44.4
Itemized Deduction ($b),1206.7,"1,193.0","1,105.2",7.9
Standard Deduction Filers (#m),117.4,85.2,107.7,-20.9
Standard Deduction ($b),876.2,740.9,925.2,-19.9
Personal Exemption ($b),1121.6,"1,110.2","1,114.0",-0.3
Taxable Income ($b),6997.9,"5,783.4","7,341.8",-21.2
Regular Tax ($b),NaN,"1,078.6","1,534.2",-29.7
AMT Income ($b),NaN,"7,751.2","9,485.9",-18.3


In [101]:
(diag_data['% Change']['Itemizers (#m)'] + diag_data['% Change']['Itemized Deduction ($b)'] +
 diag_data['% Change']['Standard Deduction Filers (#m)'] + diag_data['% Change']['Standard Deduction ($b)'])

11.482872793700693

In [102]:
# Total value of missing itemized deductions
in_billions = 1e-9
state = (calc_puf.records.e18400 * calc_puf.records.s006).sum()
print 'State and Local: {} ($b)'.format(round(state * in_billions, 2))
int_paid = (calc_puf.records.e19200 * calc_puf.records.s006).sum()
print 'Interest Paid: {} ($b)'.format(round(int_paid * in_billions, 2))
net_cas = (calc_puf.records.g20500 * calc_puf.records.s006).sum()
print 'Net Casualty or Theft Loss: {} ($b)'.format(round(net_cas * in_billions, 2))
print '-----------------'
print 'Total: {} ($b)'.format((round((state + int_paid + net_cas) * in_billions, 2)))

State and Local: 321.94 ($b)
Interest Paid: 325.15 ($b)
Net Casualty or Theft Loss: 4.62 ($b)
-----------------
Total: 651.72 ($b)


### Income Levels

In [103]:
inc_dict = OrderedDict()
inc_dict['CPS'] = [] 
inc_dict['PUF'] = []
inc_dict['Diff'] = []
inc_dict['Pct Diff'] = []
inc_list = ['WAS', 'Taxable Interest', 'Ordinary Dividends', 'Qualified Dividends', 'Business Income']
was_cps = (calc_cps.records.e00200 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(was_cps)
was_puf = (calc_puf.records.e00200 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(was_puf)
was_diff = (was_cps - was_puf)
inc_dict['Diff'].append(was_diff)
inc_dict['Pct Diff'].append((was_diff / was_puf) * 100)

int_cps = (calc_cps.records.e00300 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(int_cps)
int_puf = (calc_puf.records.e00300 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(int_puf)
int_diff = (int_cps - int_puf)
inc_dict['Diff'].append(int_diff)
inc_dict['Pct Diff'].append((int_diff / int_puf) * 100)

odiv_cps = (calc_cps.records.e00600 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(odiv_cps)
odiv_puf = (calc_puf.records.e00600 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(odiv_puf)
odiv_diff = (odiv_cps - odiv_puf)
inc_dict['Diff'].append(odiv_diff)
inc_dict['Pct Diff'].append((odiv_diff / odiv_puf) * 100)

qdiv_cps = (calc_cps.records.e00650 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(qdiv_cps)
qdiv_puf = (calc_puf.records.e00650 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(qdiv_puf)
qdiv_diff = (qdiv_cps - qdiv_puf)
inc_dict['Diff'].append(qdiv_diff)
inc_dict['Pct Diff'].append((qdiv_diff / qdiv_puf) * 100)

biz_cps = (calc_cps.records.e00900 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(biz_cps)
biz_puf = (calc_puf.records.e00900 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(biz_puf)
biz_diff = (biz_cps - biz_puf)
inc_dict['Diff'].append(biz_diff)
inc_dict['Pct Diff'].append((biz_diff / biz_puf) * 100)

inc_df = pd.DataFrame.from_dict(inc_dict)
inc_df.index = inc_list
inc_df

,CPS,PUF,Diff,Pct Diff
WAS,"6,648,838,107,939.7","6,820,912,371,695.9","-172,074,263,756.2",-2.5
Taxable Interest,"91,960,641,825.4","97,246,196,658.4","-5,285,554,833.1",-5.4
Ordinary Dividends,"245,424,347,487.0","270,137,086,776.2","-24,712,739,289.2",-9.1
Qualified Dividends,"185,442,636,961.2","198,124,994,913.5","-12,682,357,952.3",-6.4
Business Income,"309,555,316,495.2","309,444,223,964.1","111,092,531.1",0.0


### Distribution of Income Variables

In [104]:
# Generate data for distribution plots
cps_dist = pd.DataFrame()
puf_dist = pd.DataFrame()
cps_was = distribution(calc_cps.records.e00200, calc_cps.records.s006, calc_cps.records.c00100)
puf_was = distribution(calc_puf.records.e00200, calc_puf.records.s006, calc_puf.records.c00100)
cps_int = distribution(calc_cps.records.e00300, calc_cps.records.s006, calc_cps.records.c00100)
puf_int = distribution(calc_puf.records.e00300, calc_puf.records.s006, calc_puf.records.c00100)
cps_odiv = distribution(calc_cps.records.e00600, calc_cps.records.s006, calc_cps.records.c00100)
puf_odiv = distribution(calc_puf.records.e00600, calc_puf.records.s006, calc_puf.records.c00100)
cps_qdiv = distribution(calc_cps.records.e00650, calc_cps.records.s006, calc_cps.records.c00100)
puf_qdiv = distribution(calc_puf.records.e00650, calc_puf.records.s006, calc_puf.records.c00100)
cps_biz = distribution(calc_cps.records.e00900, calc_cps.records.s006, calc_cps.records.c00100)
puf_biz = distribution(calc_puf.records.e00900, calc_puf.records.s006, calc_puf.records.c00100)
cps_dist['WAS'] = cps_was[1]
puf_dist['WAS'] = puf_was[1]
cps_dist['INT'] = cps_int[1]
puf_dist['INT'] = puf_int[1]
cps_dist['ODIV'] = cps_odiv[1]
puf_dist['ODIV'] = puf_odiv[1]
cps_dist['QDIV'] = cps_qdiv[1]
puf_dist['QDIV'] = puf_qdiv[1]
cps_dist['BIZ'] = cps_biz[1]
puf_dist['BIZ'] = puf_biz[1]
cps_dist['AGI Bin'] = index_list()
puf_dist['AGI Bin'] = index_list()
cps_dist['label'] = 'CPS'
puf_dist['label'] = 'PUF'
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
soi_dist = pd.DataFrame()
for item in items_tups:
    soi_dist[item[0]] = (soi_income[item[0]] / soi_income[item[0]].sum()) * 100
soi_dist['AGI Bin'] = index_list()
soi_dist['label'] = 'SOI'
scatter_data = pd.concat([cps_dist, puf_dist, soi_dist])
scatter_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Percent of Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(scatter_data, x='AGI Bin', y=item[0], color='label', ylabel='Percent',
                      title=title, tooltips=[('PCT', '@{}'.format(item[0]))])
    scatter_list.append(scatter)

In [105]:
show(column(scatter_list))

In [106]:
cps_tot = pd.DataFrame()
puf_tot = pd.DataFrame()
cps_tot['WAS'] = cps_was[0]
puf_tot['WAS'] = puf_was[0]
cps_tot['INT'] = cps_int[0]
puf_tot['INT'] = puf_int[0]
cps_tot['ODIV'] = cps_odiv[0]
puf_tot['ODIV'] = puf_odiv[0]
cps_tot['QDIV'] = cps_qdiv[0]
puf_tot['QDIV'] = puf_qdiv[0]
cps_tot['BIZ'] = cps_biz[0]
puf_tot['BIZ'] = puf_biz[0]
cps_tot['AGI Bin'] = index_list()
puf_tot['AGI Bin'] = index_list()
cps_tot['label'] = 'CPS'
puf_tot['label'] = 'PUF'
soi_income['AGI Bin'] = index_list()
soi_income['label'] = 'SOI'
total_data = pd.concat([cps_tot, puf_tot, soi_income])
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
total_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(total_data, x='AGI Bin', y=item[0], color='label', ylabel='Total',
                      title=title, tooltips=[('Total', '@{}'.format(item[0]))])
    total_list.append(scatter)

In [107]:
show(column(total_list))

### Itemized Deduction Amounts

In [108]:
deductions_cps = {'Medical Expenses': (calc_cps.records.e17500[calc_cps.records.c04470 > 0] *
                                       calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'State and Local Taxes':  (calc_cps.records.e18400[calc_cps.records.c04470 > 0] *
                                             calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Real Estate Taxes':  (calc_cps.records.e18500[calc_cps.records.c04470 > 0] *
                                         calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Interest Paid':  (calc_cps.records.e19200[calc_cps.records.c04470 > 0] *
                                     calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Charitable Cash Contributions': (calc_cps.records.e19800[calc_cps.records.c04470 > 0] *
                                                    calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Charitable Non-Cash Contributions': (calc_cps.records.e20100[calc_cps.records.c04470 > 0] *
                                                        calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Total Misc. Expenses':  (calc_cps.records.e20400[calc_cps.records.c04470 > 0] *
                                            calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Net Casualty or Loss': (calc_cps.records.g20500[calc_cps.records.c04470 > 0] *
                                           calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum()}
ded_cps_df = pd.DataFrame.from_dict(deductions_cps, 'index')
ded_cps_df.columns = ['Total']
ded_cps_df['source'] = 'CPS'

deductions_puf = {'Medical Expenses': (calc_puf.records.e17500[calc_puf.records.c04470 > 0] *
                                       calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'State and Local Taxes':  (calc_puf.records.e18400[calc_puf.records.c04470 > 0] *
                                             calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Real Estate Taxes':  (calc_puf.records.e18500[calc_puf.records.c04470 > 0] *
                                         calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Interest Paid':  (calc_puf.records.e19200[calc_puf.records.c04470 > 0] *
                                     calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Charitable Cash Contributions': (calc_puf.records.e19800[calc_puf.records.c04470 > 0] *
                                                    calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Charitable Non-Cash Contributions': (calc_puf.records.e20100[calc_puf.records.c04470 > 0] *
                                                        calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Total Misc. Expenses':  (calc_puf.records.e20400[calc_puf.records.c04470 > 0] *
                                            calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Net Casualty or Loss': (calc_puf.records.g20500[calc_puf.records.c04470 > 0] *
                                           calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum()}
ded_puf_df = pd.DataFrame.from_dict(deductions_puf, 'index')
ded_puf_df.columns = ['Total']
ded_puf_df['source'] = 'PUF'
soi_deductions['source'] = 'SOI'

ded_full_df = pd.concat([ded_cps_df, ded_puf_df, soi_deductions])
ded_full_df['index'] = ded_full_df.index

In [109]:
(calc_puf.records.e19200[calc_puf.records.c04470 > 0] *
                                     calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum()

300472936805.24146

In [110]:
ded_bar = Bar(ded_full_df, 'index', 'Total', group='source', title='Itemized Deduction Totals',
              xlabel='Deduction', ylabel='Total', tooltips=[('Deduction', '@index'), ('Total', '@height{0.2f}'),
                                                            ('Data', '@source')])
show(ded_bar)

In [111]:
ded_error_df = pd.DataFrame()
ded_error_df['Difference - CPS'] = ded_cps_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference - CPS'] = 100 * ded_error_df['Difference - CPS'] / soi_deductions['Total']
ded_error_df['Difference - PUF'] = ded_puf_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference - PUF'] = 100 * ded_error_df['Difference - PUF'] / soi_deductions['Total']
print 'Error in Itemized Deductions Relative to SOI Totals'
ded_error_df

Error in Itemized Deductions Relative to SOI Totals


,Difference - CPS,% Difference - CPS,Difference - PUF,% Difference - PUF
Charitable Cash Contributions,"6,904,717,936.3",4.4,"-4,703,427,583.1",-3.0
Charitable Non-Cash Contributions,"-29,721,920,506.2",-45.5,"-31,555,102,263.5",-48.3
Interest Paid,"106,631,024,260.8",34.6,"-7,489,448,194.8",-2.4
Medical Expenses,"-15,965,701,434.7",-12.4,"-9,779,156,210.6",-7.6
Net Casualty or Loss,"-2,204,349,000.0",-100.0,"2,251,393,716.6",102.1
Real Estate Taxes,"-51,867,209,952.4",-28.7,"17,247,882,146.1",9.5
State and Local Taxes,"-13,314,991,985.8",-4.1,"-16,248,434,290.2",-4.9
Total Misc. Expenses,"-21,972,301,017.9",-17.4,"3,957,677,605.4",3.1


In [112]:
# Create DataFrame subset of calculator records for the CPS
item_df = pd.DataFrame({'e00200': calc_cps.records.e00200,
                        's006': calc_cps.records.s006,
                        'c04470': calc_cps.records.c04470})
item_df['itemizer'] = np.where(calc_cps.records.c04470 > 0, 1, 0)
item_participation_rt = percentile(item_df, 'itemizer', 100, 'e00200', 's006')
# Create DataFrame subset for PUF
item_df_puf = pd.DataFrame({'e00200': calc_puf.records.e00200,
                            's006': calc_puf.records.s006,
                            'c04470': calc_puf.records.c04470})
item_df_puf['itemizer'] = np.where(calc_puf.records.c04470 > 0, 1, 0)
item_participation_rt_puf = percentile(item_df_puf, 'itemizer', 100, 'e00200', 's006')
item_index = item_participation_rt.index

In [113]:
hover_ded = HoverTool(tooltips=[('Percentile', '$index'), ('Itemizing Rate', '$x')])
f = figure(title='Itemizing Rate by Wage Percentile',
           x_axis_label='Wage Percentile', tools=[hover_ded, 'save', 'reset', 'zoom_in', 'zoom_out'])
f.line(item_index, item_participation_rt, legend='CPS')
f.line(item_index, item_participation_rt_puf, legend='PUF', color='red')
f.legend.location = 'top_left'
show(f)

In [114]:
# Average Itemized Deduction by percentile
itemizers_cps = copy.deepcopy(item_df[item_df['c04470'] > 0])
item_mean_cps = percentile(itemizers_cps, 'c04470', 100, 'e00200', 's006')
itemizers_puf = copy.deepcopy(item_df_puf[item_df_puf['c04470'] > 0])
item_mean_puf = percentile(itemizers_puf, 'c04470', 100, 'e00200', 's006')

In [115]:
hover = HoverTool(tooltips=[('Percentile', '$index'), ('Mean', '$y')])
f = figure(title='Mean Itemized Deduction Total Among Itemizers',
           x_axis_label='Wage Percentile', tools=[hover, 'save', 'reset', 'zoom_in', 'zoom_out'])
f.line(item_index, item_mean_cps, legend='CPS')
f.line(item_index, item_mean_puf, legend='PUF', color='red')
f.legend.location = 'top_left'
show(f)

### Refundable Credits

In [116]:
eitc_cps = (calc_cps.records.eitc * calc_cps.records.s006).sum()
c11070_cps = (calc_cps.records.c11070 * calc_cps.records.s006).sum()
c10960_cps = (calc_cps.records.c10960 * calc_cps.records.s006).sum()
personal_credit_cps = (calc_cps.records.personal_credit * calc_cps.records.s006).sum()
ctc_new_cps = (calc_cps.records.ctc_new * calc_cps.records.s006).sum()

eitc_puf = (calc_puf.records.eitc * calc_puf.records.s006).sum()
c11070_puf = (calc_puf.records.c11070 * calc_puf.records.s006).sum()
c10960_puf = (calc_puf.records.c10960 * calc_puf.records.s006).sum()
personal_credit_puf = (calc_puf.records.personal_credit * calc_puf.records.s006).sum()
ctc_new_puf = (calc_puf.records.ctc_new * calc_puf.records.s006).sum()

In [117]:
print 'Refundable Credits'
pd.DataFrame(OrderedDict({'PUF': [eitc_puf, c11070_puf, c10960_puf, personal_credit_puf, ctc_new_puf],
                          'CPS': [eitc_cps, c11070_cps, c10960_cps, personal_credit_cps, ctc_new_cps],
                          'Diff': [eitc_cps - eitc_puf, c11070_cps - c11070_puf, c10960_cps - c10960_puf,
                                   personal_credit_cps - personal_credit_puf,
                                   ctc_new_cps - ctc_new_puf]}),
                          index=['eitc', 'c11070', 'c10960', 'personal credit', 'ctc new'])

Refundable Credits


,Diff,PUF,CPS
eitc,"-6,988,514,198.6","65,207,453,506.9","58,218,939,308.3"
c11070,"-4,283,677,948.7","24,274,900,731.4","19,991,222,782.7"
c10960,"-8,382,574,969.7","8,382,574,969.7",0.0
personal credit,0.0,0.0,0.0
ctc new,0.0,0.0,0.0


In [118]:
eic0cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 0])
eic1cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 1])
eic2cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 2])
eic3cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 3])
eic0puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 0])
eic1puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 1])
eic2puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 2])
eic3puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 3]) 
eic_tot_cps = calc_cps.records.s006.sum()
eic_tot_puf = calc_puf.records.s006.sum()
print 'Percent and Number of Tax Units with Specified Number of EIC Qualified Children'
pd.DataFrame({'CPS': [eic0cps, eic1cps, eic2cps, eic3cps],
              'PUF': [eic0puf, eic1puf, eic2puf, eic3puf],
              '% - CPS': [eic0cps / eic_tot_cps, eic1cps / eic_tot_cps,
                          eic2cps / eic_tot_cps, eic3cps / eic_tot_cps],
              '% - PUF': [eic0puf / eic_tot_puf, eic1puf / eic_tot_puf,
                          eic2puf / eic_tot_puf, eic3puf / eic_tot_puf]})

Percent and Number of Tax Units with Specified Number of EIC Qualified Children


,% - CPS,% - PUF,CPS,PUF
0,0.7,0.9,"114,745,569.5","140,952,059.1"
1,0.1,0.1,"22,983,420.2","10,061,670.4"
2,0.1,0.0,"16,819,448.2","7,931,481.3"
3,0.1,0.0,"8,647,332.5","3,279,408.9"


### Benefit Programs

In [119]:
# Columns used for participation rates
cps['ssi_part'] = np.where(cps.ssi_ben > 0, 1, 0)
cps['snap_part'] = np.where(cps.snap_ben > 0, 1, 0)
cps['mcare_part'] = np.where(cps.mcare_ben > 0, 1, 0)
cps['mcaid_part'] = np.where(cps.mcaid_ben > 0, 1, 0)
cps['ss_part'] = np.where(cps.ss_ben > 0, 1, 0)
cps['vb_part'] = np.where(cps.vet_ben > 0, 1, 0)

In [120]:
benefits = pd.DataFrame(OrderedDict({'SSI': [(cps.ssi_ben * cps.s006).sum()],
                                     'SNAP': [(cps.snap_ben * cps.s006).sum()],
                                     'Medicare': [(cps.mcare_ben * cps.s006).sum()],
                                     'Medicaid': [(cps.mcaid_ben * cps.s006).sum()],
                                     'Social Security': [(cps.ss_ben * cps.s006).sum()],
                                     'VB': [(cps.vet_ben * cps.s006).sum()]})).transpose()
benefits.columns = ['Total']
benefits['Participants'] = [cps.s006[cps.ss_ben > 0].sum(), cps.s006[cps.vet_ben > 0].sum(),
                            cps.s006[cps.mcare_ben > 0].sum(), cps.s006[cps.mcaid_ben > 0].sum(),
                            cps.s006[cps.snap_ben > 0].sum(), cps.s006[cps.ssi_ben > 0].sum()]
benefits['Avg. Benefit Per Person'] = benefits['Total'] / benefits['Participants']
print 'Benefits Totals'
benefits

Benefits Totals


,Total,Participants,Avg. Benefit Per Person
Social Security,"630,068,059,192.4","33,597,757.4","18,753.3"
VB,"146,836,134,085.4","4,910,493.5","29,902.5"
Medicare,"488,236,708,639.1","38,589,780.8","12,652.0"
Medicaid,"352,985,014,221.6","27,937,147.8","12,635.0"
SNAP,"82,336,140,621.7","28,322,529.5","2,907.1"
SSI,"54,186,915,454.5","7,276,536.7","7,446.8"


#### Benefit Program Participation Rates

In [121]:
benefit_participation_names = [('ssi_part', 'SSI'), ('snap_part', 'SNAP'), ('mcare_part', 'Medicare'),
                               ('mcaid_part', 'Medicaid'), ('ss_part', 'Social Security'),
                               ('vb_part', "Veteran's Benefits")]

In [122]:
# Create list of bokeh figure displaying participation rates in each program
fig_list = list()
for item in benefit_participation_names:
    # Add Hover tools
    hover_ben = HoverTool(tooltips=[('Participation Rate', '$y{0.2f}'), ('Wage Percentile', '$x')])
    wcps = percentile(cps, item[0], 100, 'e00200', 's006')
    f = figure(title='{} Participation Rate'.format(item[1]),
               x_axis_label='Wage Percentile', tools=[hover_ben])
    f.line(wcps.index, wcps)
    fig_list.append(f)

In [123]:
show(column(fig_list))

#### Average Benefits Received by Participants

In [124]:
benefit_program_names = [('ssi_ben', 'SSI'), ('snap_ben', 'SNAP'), ('mcare_ben', 'Medicare'),
                         ('mcaid_ben', 'Medicaid'), ('ss_ben', 'Social Security'),
                         ('vet_ben', "Veteran's Benefits")]

In [125]:
fig_list = list()
for item in benefit_program_names:
    # Only taking the average benefit of participants
    bcps = copy.deepcopy(cps[cps[item[0]] > 0])
    # Add Hover tools
    hover_ben2 = HoverTool(tooltips=[('Avg. Benefit', '$y{0.2f}'), ('Wage Percentile', '$x')])
    wcps = percentile(bcps, item[0], 100, 'e00200', 's006')
    f = figure(title='Average {} Benefit - Participants'.format(item[1]),
               x_axis_label='Wage Percentile', y_axis_label='Avg. Benefit',
               tools=[hover_ben2])
    f.line(wcps.index, wcps)
    fig_list.append(f)

In [126]:
show(column(fig_list))

#### Average Benefits Received by Entire Population

In [127]:
fig_list = list()
for item in benefit_program_names:
    # Only taking the average benefit of participants
    wcps = percentile(cps, item[0], 100, 'e00200', 's006')
    # Add Hover tools
    hover_ben3 = HoverTool(tooltips=[('Avg. Benefit', '$y{0.2f}'), ('Wage Percentile', '$x')])
    f = figure(title='Average {} Benefit - Entire Population'.format(item[1]),
               x_axis_label='Wage Percentile', y_axis_label='Avg. Ben',
               tools=[hover_ben3])
    f.line(wcps.index, wcps)
    fig_list.append(f)

In [128]:
show(column(fig_list))